# Depressinator -

## Libraries

In [ ]:
# packages

import numpy as np
import pandas as pd
import time
import os

# plots
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences


import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.backend import expand_dims
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences



## Data loading

In [ ]:
data_path = 'C:/Users/DELL/PycharmProjects/Depression-Detector-by-Nueral-Network/Assignment/data/'

condition_dir = os.path.join(data_path, "condition")
control_dir = os.path.join(data_path, "control")
scores_file = os.path.join(data_path, "scores.csv")

conditions = [pd.read_csv(os.path.join(condition_dir, filename)) for filename in os.listdir(condition_dir)]

controls = [pd.read_csv(os.path.join(control_dir, filename)) for filename in os.listdir(control_dir)]

scores_data = pd.read_csv(scores_file)


## Condition Group

In [ ]:
ymax = 8500

# iterate over all files and plot
for i in range(23):
    j = 1+i
    my_file = data_path + 'condition/condition_' + str(j) + '.csv'
    df_temp = pd.read_csv(my_file)

    # plot full activity time series
    my_alpha=0.25
    fig, ax = plt.subplots(figsize=(18,6))
    ax.scatter(df_temp.timestamp, df_temp.activity , alpha=my_alpha)
    ax.xaxis.set_major_locator(plt.MaxNLocator(20)) # reduce number of x-axis labels
    ax.set_ylim(0, ymax)
    plt.title(my_file)
    plt.xticks(rotation=90)
    plt.grid()
    ax.legend(loc='upper left')
    plt.show()

## Control Group

In [ ]:
ymax = 8500

# iterate over all files and plot
for i in range(23):
    j = 1+i
    my_file = data_path + 'control/control_' + str(j) + '.csv'
    df_temp = pd.read_csv(my_file)

    # plot full activity time series
    my_alpha=0.25
    fig, ax = plt.subplots(figsize=(18,6))
    ax.scatter(df_temp.timestamp, df_temp.activity , alpha=my_alpha)
    ax.xaxis.set_major_locator(plt.MaxNLocator(20)) # reduce number of x-axis labels
    ax.set_ylim(0, ymax)
    plt.title(my_file)
    plt.xticks(rotation=90)
    plt.grid()
    ax.legend(loc='upper left')
    plt.show()

## Scores Data

In [ ]:
scores_data.head()
scores_data.dtypes

## Exploratory data analysis

### Number of patients

In [ ]:
xs = ["Condition", "Control"]
ys = [len(conditions), len(controls)]

print(f"Condition number: {len(conditions)}")
print(f"Control number: {len(controls)}")

plt.bar(xs, ys)
plt.xlabel("Group")
plt.ylabel("Patients")
plt.title("Number of patients per group")
plt.show()

In [ ]:
condition_rows = pd.Series([len(df) for df in conditions])
condition_rows.describe()

control_rows = pd.Series([len(df) for df in controls])
control_rows.describe()

### Columns and types

In [ ]:
conditions[0].dtypes
controls[0].dtypes

### Missing values

In [ ]:
condition_NaNs = pd.Series([df["activity"].isna().sum() for df in conditions])
condition_NaNs.mean()

control_NaNs = pd.Series([df["activity"].isna().sum() for df in controls])
control_NaNs.mean()

## Training Model

In [ ]:
X = []
y = []

for i in range(1, 24):
	df = pd.read_csv("data\condition\condition_{}.csv".format(i))
	x1 = np.array(df['activity'].tolist())
	X.append(x1)
	y.append(1)

for i in range(1, 33):
	df = pd.read_csv("data\control\control_{}.csv".format(i))
	x1 = np.array(df['activity'].tolist())
	X.append(x1)
	y.append(0)

X = np.array(X)
y = np.array(y)

X = pad_sequences(
	X,
	maxlen=max([len(x1) for x1 in X]),
	padding='post',
	truncating='post',
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

model = models.Sequential([
	layers.Dense(128, activation='relu', input_shape=X_train.shape[1:]),
	layers.Dropout(0.3),

	layers.Dense(64, activation='relu'),
	layers.Dropout(0.3),

	layers.Dense(1, activation='sigmoid')
])

model.compile(
	optimizer="adam",
	loss="binary_crossentropy",
	metrics=["binary_accuracy"],
)

acc = 0
while(acc <= 0.80):
	history = model.fit(
		X_train,
		y_train,
		validation_data=(X_test, y_test),
		batch_size=60,
		epochs=1,
	)
	acc = history.history['val_binary_accuracy'][-1]

model.save("depression.h5")

## Detect Depression

In [ ]:
model = tf.keras.models.load_model("depression.h5")

def deep_depression_detector(activity_data_csv_file):
	df = pd.read_csv(activity_data_csv_file)
	x = np.array([df['activity'].tolist()])
	x = pad_sequences(
		x, maxlen=65407,
		padding='post',
		truncating='post'
	)

	y_pred = model.predict(x)
	if y_pred >= 0.5:
		return  {'prediction':'depressed'}
	else:
		return  {'prediction':'nondepressed'}